In [60]:
# BFS with levels
# Runtime: 312 ms, faster than 24.66%
# https://leetcode.com/problems/remove-invalid-parentheses/discuss/75028/Short-Python-BFS
from typing import List
class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        def isvalid(s: str) -> bool:
            stack = 0
            for ch in s:
                if ch == '(': 
                    stack += 1
                elif ch == ')':
                    stack -= 1
                    if stack < 0: return False
            return stack == 0
        
        lvl = {s}
        while True:
            valid = list(filter(isvalid, lvl))
            if valid: # return minimum removals at the same level
                return valid
            newset = set()
            for st in lvl:
                for i in range(len(st)):
                    # Remove parenthesis at index i
                    newset.add(st[:i] + st[i+1:])
            lvl = newset
#             lvl = {s[:i] + s[i+1:] for s in lvl for i in range(len(s))}

In [63]:
# BFS with no levels (naive solution with hashset)
# Runtime: 436 ms, faster than 19.00%
# https://leetcode.com/problems/remove-invalid-parentheses/discuss/75038/Evolve-from-intuitive-solution-to-optimal-a-review-of-all-solutions
from typing import List
class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        def isvalid(s: str) -> bool:
            stack = 0
            for ch in s:
                if ch == '(': 
                    stack += 1
                elif ch == ')':
                    stack -= 1
                    if stack < 0: return False
            return stack == 0
        
        q = [s]
        seen = set()
        res = []
        while q:
            st = q.pop(0)
            if st in seen: continue
            seen.add(st)
            if isvalid(st):
                res.append(st)
            elif not res:
                for i, ch in enumerate(st):
                    if ch == ')' or ch == '(':
                        q.append(st[:i] + st[i+1:])
        return res

In [41]:
# BFS - suboptimal solution
# Avoid two duplicates:
# 1. due to removing the same set of characters in different order.
#    So, we can enforce an order by keeping the last removal index and remove after it only.
# 2. handling consecutive same chars. We can just remove the 1st one.

# Runtime: 140 ms, faster than 51.19%
from typing import List
class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        def isvalid(s: str) -> bool:
            stack = 0
            for ch in s:
                if ch == '(': 
                    stack += 1
                elif ch == ')':
                    stack -= 1
                    if stack < 0: return False
            return stack == 0
        
        q = [(s,0)]
        res = []
        while q:
            st, last_rm_idx = q.pop(0)
            if isvalid(st):
                res.append(st)
            else:
                if not res:
                    # 1. traverse from last_rm_idx to end of st to avoid different order duplicates
                    for i in range(last_rm_idx, len(st)):
                        # 2. always remove the first one when met consecutive ones
                        if (st[i] == ')' or st[i] == '(') and (i == last_rm_idx or st[i] != st[i-1]):
                            q.append((st[:i] + st[i+1:], i))
        return res

In [125]:
# core function reused in BFS/DFS optimal solutions
from typing import Callable
def scan_and_remove(ss: str, last_i: int, last_j: int, pair_l: str, res: List[str], f:Callable=None):
    stack = 0
    pair_r = ['(',')'][pair_l=='('] # get the other pair
#     print(ss, "i=",last_i," j=",last_j)
    for i in range(last_i, len(ss)):
        if ss[i] == pair_l: stack += 1
        elif ss[i] == pair_r: stack -= 1
        if stack >= 0: continue # still valid
        # last_j is last_remove_index
        for j in range(last_j, i+1):
            if ss[j] == pair_r and (j == last_j or ss[j-1] != pair_r): # seek first pair_r
                # remove j-th
                if f:
                    f(ss[:j]+ss[j+1:], i, j, pair_l)
                else:
                    scan_and_remove(ss[:j]+ss[j+1:], i, j, pair_l, res)
        break
    if stack >= 0:
        sr = ss[::-1]
        if pair_l == '(': # left to right finished, do right to left
            if f:
                f(sr, 0, 0, ')')
            else:
                scan_and_remove(sr, 0, 0, ')', res)
        else: # right to left finished
            res.append(sr)

In [134]:
# BFS - optimal solution
# Runtime: 40 ms, faster than 94.35%
from typing import List
class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        q = [(s,0,0,'(')]
        res = []
        while q:
            ele = q.pop(0)
            ss, last_i, last_j, pair_l = ele
            funcAppend = lambda _1,_2,_3,_4: q.append((_1,_2,_3,_4))
            scan_and_remove(ss, last_i, last_j, pair_l, res, funcAppend)
        return res

In [136]:
# DFS - optimal solution
# Runtime: 40 ms, faster than 94.20%

#* remove-invalid-parentheses/discuss/75027/Easy-Short-Concise-and-Fast-Java-DFS-3-ms-solution
# To make the prefix valid, we need to remove a ‘)', any one!
# And, we always remove the first ) in concecutive ')'s.
# And, what about redudant '('s? Reverse the string and do the same!
from typing import List
class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        res = []
        scan_and_remove(s, 0, 0, '(', res)
        return res

In [137]:
Solution().removeInvalidParentheses("()())()")

['(())()', '()()()']

In [128]:
Solution().removeInvalidParentheses("(a)())()")

['(a())()', '(a)()()']

In [129]:
Solution().removeInvalidParentheses(")(")

['']

In [120]:
Solution().removeInvalidParentheses("a)b)c)d)e)f)g)h)i)j)((((((((((((")

['abcdefghij']